In [1]:
!pip install gradio torch torchvision diffusers transformers accelerate scipy safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/

In [2]:
!pip install segment-anything

In [11]:
%cd /content/drive/MyDrive/SAM_DIFFUSION/GroundingDINO/

/content/drive/MyDrive/SAM_DIFFUSION/GroundingDINO


In [4]:
!pip install -e .

Obtaining file:///content/drive/MyDrive/SAM_DIFFUSION/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 10.7 MB/s eta 0:00:00
  Running setup.py develop for groundingdino


In [14]:
import gradio as gr
import numpy as np
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

import GroundingDINO.groundingdino.datasets.transforms as T
from GroundingDINO.groundingdino.models import build_model
from GroundingDINO.groundingdino.util import box_ops
from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from GroundingDINO.groundingdino.util.inference import annotate, load_image, predict
from huggingface_hub import hf_hub_download

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam_checkpoint = "/content/drive/MyDrive/SAM_DIFFUSION/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype = torch.float16
)
pipe = pipe.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [15]:
def load_grounding_dino_model(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file)
    model = build_model(args)
    args.device = device

    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location='cpu')
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model

In [16]:
# Use this command for evaluate the Grounding DINO model
# Option2 : download the model
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filename = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"

In [17]:
groundingdino_model = load_grounding_dino_model(ckpt_repo_id, ckpt_filename, ckpt_config_filename)
groundingdino_model = groundingdino_model.to(device)

GroundingDINO_SwinB.cfg.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

groundingdino_swinb_cogcoor.pth:   0%|          | 0.00/938M [00:00<?, ?B/s]

Model loaded from /root/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [18]:
# selected_pixels = []
# # Inpainting Quality Metrics
# from skimage.metrics import structural_similarity as compare_ssim

# with gr.Blocks() as demo:
#     with gr.Row():
#         input_img = gr.Image(label="Input")
#         mask_img = gr.Image(label="Mask")
#         output_img = gr.Image(label="Output")

#     with gr.Blocks():
#         prompt_text = gr.Textbox(lines=1, label="Prompt")

#     with gr.Row():
#         submit =gr.Button("submit")

#     # SSIM
#     def ssim(img1, img2):
#         img1_gray = cv2.cvtColor(np.array(img1), cv2.COLOR_RGB2GRAY)
#         img2_gray = cv2.cvtColor(np.array(img2), cv2.COLOR_RGB2GRAY)
#         ssim_score, _ = compare_ssim(img1_gray, img2_gray, full=True)
#         return ssim_score

#     # PSNR
#     def psnr(img1, img2):
#         img1 = np.array(img1)
#         img2 = np.array(img2)
#         mse = np.mean((np.array(img1, dtype=np.float32) - np.array(img2, dtype=np.float32)) ** 2)
#         if mse == 0:
#             return float('inf')
#         PIXEL_MAX = 255.0
#         return 30 * np.log10(PIXEL_MAX / np.sqrt(mse))

#     def denoising(image):
#         # Gaussian blur
#         if not isinstance(image, np.ndarray):
#             image = np.array(image)
#         return cv2.GaussianBlur(image, (5, 5), 0)

#     def edge_enhancement(image):
#         # Laplacian edge detection followed by sharpening
#         if not isinstance(image, np.ndarray):
#             image = np.array(image)

#         blurred = cv2.GaussianBlur(image, (3, 3), 0)
#         edges = cv2.Laplacian(blurred, cv2.CV_64F)
#         sharpened = image - 0.7 * edges
#         sharpened[sharpened < 0] = 0
#         sharpened[sharpened > 255] = 255
#         return sharpened.astype(np.uint8)

#     def generate_mask(image, evt:gr.SelectData):
#         selected_pixels.append(evt.index)
#         predictor.set_image(image)
#         input_points = np.array(selected_pixels)
#         input_labels = np.ones(input_points.shape[0])
#         mask, _, _ = predictor.predict(
#             point_coords=input_points,
#             point_labels=input_labels,
#             multimask_output=False
#         )
#         #(n,size,size)
#         # mask = np.logical_not(mask) # for selection of foregraound and background will change.
#         mask = Image.fromarray(mask[0, :, :])
#         return mask

#     def inpaint(image, mask, prompt):
#         image = Image.fromarray(image)
#         mask = Image.fromarray(mask)

#         image = image.resize((512,512))
#         mask = mask.resize((512,512))

#         output = pipe(
#             prompt = prompt,
#             image= image,
#             mask_image = mask
#         ).images[0]

#         # Compute PSNR and SSIM for the output
#         psnr_score = psnr(image, output)
#         ssim_score = ssim(image, output)

#         # Post-processing techniques
#         output = denoising(output)
#         output = edge_enhancement(output)

#         return output, ssim_score

#     # Process input image using detect_boxes function
#     bbox_img.process(detect_boxes, inputs=[input_img])

#     input_img.select(generate_mask, [input_img],[mask_img])
#     submit.click(inpaint, inputs=[input_img,mask_img,prompt_text], outputs=[output_img, gr.Label("SSIM")])

# if __name__ == "__main__":
#     demo.launch(share=True, debug = True)

In [24]:
selected_pixels = []
# Inpainting Quality Metrics
from skimage.metrics import structural_similarity as compare_ssim

with gr.Blocks() as demo:
    with gr.Row():
        input_img = gr.Image(label="Input")
        bbox_img = gr.Image(label="DINO Object Detection")
        mask_img = gr.Image(label="Mask")
        output_img = gr.Image(label="Output")

    with gr.Blocks():
        prompt_text_detect = gr.Textbox(lines=1, label="Detection Prompt")
        prompt_text = gr.Textbox(lines=1, label="Inpainting Prompt")

    with gr.Row():
        submit = gr.Button("submit")

    def run_grounding(input_image, grounding_caption, box_threshold=0.5, text_threshold=0.25):
        ##DINO
        init_image_pil = Image.fromarray(input_image)
        init_image = init_image_pil.convert("RGB")

        transform = T.Compose([
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        transformed_image, _ = transform(init_image, None)

        # Move tensors to the same device within run_grounding_and_sam function
        # transformed_image = transformed_image.to(device)

        # run grounding
        boxes, logits, phrases = predict(
            model=groundingdino_model,
            image=transformed_image,
            caption=grounding_caption,
            box_threshold=box_threshold,
            text_threshold=text_threshold
        )

        # Move boxes tensor to the same device
        # boxes = boxes.to(device)

        # Convert the image to a NumPy array
        image_source= np.array(init_image)

        annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
        annotated_frame = annotated_frame[...,::-1]

        # # To Visualize DINO Detection
        image_with_box = Image.fromarray(annotated_frame)
        return image_with_box

    # SSIM
    def ssim(img1, img2):
        img1_gray = cv2.cvtColor(np.array(img1), cv2.COLOR_RGB2GRAY)
        img2_gray = cv2.cvtColor(np.array(img2), cv2.COLOR_RGB2GRAY)
        ssim_score, _ = compare_ssim(img1_gray, img2_gray, full=True)
        return ssim_score

    # PSNR
    def psnr(img1, img2):
        img1 = np.array(img1)
        img2 = np.array(img2)
        mse = np.mean((np.array(img1, dtype=np.float32) - np.array(img2, dtype=np.float32)) ** 2)
        if mse == 0:
            return float('inf')
        PIXEL_MAX = 255.0
        return 30 * np.log10(PIXEL_MAX / np.sqrt(mse))

    def denoising(image):
        # Gaussian blur
        if not isinstance(image, np.ndarray):
            image = np.array(image)
        return cv2.GaussianBlur(image, (5, 5), 0)

    def edge_enhancement(image):
        # Laplacian edge detection followed by sharpening
        if not isinstance(image, np.ndarray):
            image = np.array(image)

        blurred = cv2.GaussianBlur(image, (3, 3), 0)
        edges = cv2.Laplacian(blurred, cv2.CV_64F)
        sharpened = image - 0.7 * edges
        sharpened[sharpened < 0] = 0
        sharpened[sharpened > 255] = 255
        return sharpened.astype(np.uint8)

    def generate_mask(image, evt:gr.SelectData):
        selected_pixels.append(evt.index)
        predictor.set_image(image)
        input_points = np.array(selected_pixels)
        input_labels = np.ones(input_points.shape[0])
        mask, _, _ = predictor.predict(
            point_coords=input_points,
            point_labels=input_labels,
            multimask_output=False
        )
        #(n,size,size)
        # mask = np.logical_not(mask) # for selection of foregraound and background will change.
        mask = Image.fromarray(mask[0, :, :])
        return mask

    def inpaint(image, mask, prompt):
        image = Image.fromarray(image)
        mask = Image.fromarray(mask)

        image = image.resize((512,512))
        mask = mask.resize((512,512))

        output = pipe(
            prompt = prompt,
            image= image,
            mask_image = mask
        ).images[0]

        # Compute PSNR and SSIM for the output
        psnr_score = psnr(image, output)
        ssim_score = ssim(image, output)

        # Post-processing techniques
        output = denoising(output)
        output = edge_enhancement(output)

        return output, ssim_score


    submit.click(run_grounding, inputs=[input_img,prompt_text_detect], outputs=[bbox_img])

    bbox_img.select(generate_mask, [bbox_img],[mask_img])
    submit.click(inpaint, inputs=[input_img,mask_img,prompt_text], outputs=[output_img, gr.Label("SSIM")])

if __name__ == "__main__":
    demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4ebfff033bd5abac91.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:962: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/

  0%|          | 0/50 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4ebfff033bd5abac91.gradio.live
